1. 가장 복잡한 예제 요청
2. JSON 전달해서 구동되는지 확인 부탁
   다르게 생긴 C++ 코드도 돌아가는지 확인하기 위해서 
3. Jupyter notebook에서 C++ 코드용 CELL을 표기하는 특수 함수 만들기
4. cp.compile_from_source 기능 추가해서 디사일로에 제안

In [1]:
from compyler import cpp_loader

# 1. Runing C++ source code 

In [2]:
extracted_function = cpp_loader.load_cpp_fun("./source_cpp/example1.cpp", function_name='fun')
print(extracted_function)

No syntax errors found.
Ciphertext fun(Ciphertext a) {
    // Create FHE circuit
    auto b = -a + 2;
    auto c = a - b;

    return c;
}


In [3]:
extracted_function = cpp_loader.load_cpp_fun("./source_cpp/example1.cpp", function_name='main')
print(extracted_function)

3 <SourceLocation file 'temp.cpp', line 2, column 5>: unknown type name 'FheProgram'
1 issue(s) found.
None


In [2]:
extracted_function = cpp_loader.load_cpp_fun("./source_cpp/example1.cpp", function_name='fun_err')
print(extracted_function)

ValueError: Function 'fun_err' not found in the file.

In [3]:
path ="./source_cpp/example2.cpp"
cpp_loader.compile_cpp(path, function_name='fun')

No syntax errors found.
Full Python code:

import compyler as cp
program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40)
    # Create FHE circuit
b = -a + 2
c = a
for i in range(0, 5, 1):
    c = c * (-c + 2)
    b = b * (-c + 2)
d = b.mean() + c.mean()
program.cppcompile(d)

Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json


# 2. On-line C++ editing 

In [2]:
from IPython.core.magic import register_cell_magic
from IPython.display import Markdown, display

@register_cell_magic
def cpp(line, cell):
    # Extract the variable name from the line argument
    variable_name = line.strip()
    
    # Store the C++ code in the specified variable
    if variable_name:
        globals()[variable_name] = cell
    
    # Display the code block with C++ syntax highlighting
    display(Markdown(f"```c++\n{cell}\n```"))

1. "%%cpp" 라는 특수 함수를 만들어서 Jupyter notebook에서 C++ 코드를 실행할 수 있게 한다.
2. "my_cpp_code" 라는 변수에 C++ 코드를 저장한다.
3. cpp_loader.

In [3]:
%%cpp my_cpp_code

Ciphertext fun(Ciphertext a) {
    auto b = -a + 2;
    auto c = a - b;

    return c;
}

```c++

Ciphertext fun(Ciphertext a) {
    auto b = -a + 2;
    auto c = a - b;

    return c;
}

```

In [4]:
cpp_loader.compile_cpp(my_cpp_code)

Full Python code:

import compyler as cp
program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40)
b = -a + 2
c = a - b
program.cppcompile(c)

Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json
